In [1]:
from sklearn import preprocessing
from PIL import Image
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import os
from sklearn.metrics import accuracy_score
from os.path import dirname


def replace_nan_with_row_mean(a):
    out = np.where(np.isnan(a), ma.array(a, mask=np.isnan(a)).mean(axis=1)[:, np.newaxis], a)
    return np.float32(out)

def replace_nan_with_near_value(a):
    mask = np.isnan(a)
    idx = np.where(~mask,np.arange(mask.shape[1]),0)
    np.maximum.accumulate(idx,axis=1, out=idx)
    out = a[np.arange(idx.shape[0])[:,None], idx]
    return np.float32(out)

def set_nan_to_zero(a):
    where_are_NaNs = np.isnan(a)
    a[where_are_NaNs] = 0
    return a






In [2]:
def TSC_data_loader(dataset_path,dataset_name):
    Train_dataset = np.loadtxt(
        dataset_path + '/' + dataset_name + '/' + dataset_name + '_TRAIN.tsv')
    Test_dataset = np.loadtxt(
        dataset_path + '/' + dataset_name + '/' + dataset_name + '_TEST.tsv')
    Train_dataset = Train_dataset.astype(np.float32)
    Test_dataset = Test_dataset.astype(np.float32)

    X_train = Train_dataset[:, 1:]
    y_train = Train_dataset[:, 0:1]

    X_test = Test_dataset[:, 1:]
    y_test = Test_dataset[:, 0:1]
    le = preprocessing.LabelEncoder()
    le.fit(np.squeeze(y_train, axis=1))
    y_train = le.transform(np.squeeze(y_train, axis=1))
    y_test = le.transform(np.squeeze(y_test, axis=1))
    return replace_nan_with_row_mean(X_train), y_train, replace_nan_with_row_mean(X_test), y_test


In [3]:
def fill_out_with_Nan(data,max_length):
    #via this it can works on more dimensional array
    pad_length = max_length-data.shape[-1]
    if pad_length == 0:
        return data
    else:
        pad_shape = list(data.shape[:-1])
        pad_shape.append(pad_length)
        Nan_pad = np.empty(pad_shape)*np.nan
        return np.concatenate((data, Nan_pad), axis=-1)
    

def get_label_dict(file_path):
    label_dict ={}
    with open(file_path) as file:
        lines = file.readlines()
        for line in lines:
            if '@classLabel' in line:
                label_list = line.replace('\n','').split(' ')[2:]
                for i in range(len(label_list)):
                    label_dict[label_list[i]] = i 
                
                break
    return label_dict


def get_data_and_label_from_ts_file(file_path,label_dict):
    with open(file_path) as file:
        lines = file.readlines()
        Start_reading_data = False
        Label_list = []
        Data_list = []
        max_length = 0
        for line in lines:
            if Start_reading_data == False:
                if '@data'in line:
                    Start_reading_data = True
            else:
                temp = line.split(':')
                Label_list.append(label_dict[temp[-1].replace('\n','')])
                data_tuple= [np.expand_dims(np.fromstring(channel, sep=','), axis=0) for channel in temp[:-1]]
                max_channel_length = 0
                for channel_data in data_tuple:
                    if channel_data.shape[-1]>max_channel_length:
                        max_channel_length = channel_data.shape[-1]
                data_tuple = [fill_out_with_Nan(data,max_channel_length) for data in data_tuple]
                data = np.expand_dims(np.concatenate(data_tuple, axis=0), axis=0)
                Data_list.append(data)
                if max_channel_length>max_length:
                    max_length = max_channel_length
        
        Data_list = [fill_out_with_Nan(data,max_length) for data in Data_list]
        X =  np.concatenate(Data_list, axis=0)
        Y =  np.asarray(Label_list)
        
        return np.float32(X), Y




def TSC_multivariate_data_loader(dataset_path, dataset_name):
    
    Train_dataset_path = dataset_path + '/' + dataset_name + '/' + dataset_name + '_TRAIN.ts'
    Test_dataset_path = dataset_path + '/' + dataset_name + '/' + dataset_name + '_TEST.ts'
    label_dict = get_label_dict(Train_dataset_path)
    X_train, y_train = get_data_and_label_from_ts_file(Train_dataset_path,label_dict)
    X_test, y_test = get_data_and_label_from_ts_file(Test_dataset_path,label_dict)
    
    return set_nan_to_zero(X_train), y_train, set_nan_to_zero(X_test), y_test





dataset_name = 'CharacterTrajectories'
dataset_path = dirname("./Example_Datasets/UEAArchive_2018/")
#TSC_multivariate_data_loader(dataset_path, dataset_name)

X_train, y_train, X_test, y_test = TSC_multivariate_data_loader(dataset_path, dataset_name)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_test.dtype)

(1422, 3, 180)
(1422,)
(1436, 3, 182)
(1436,)
int64


In [4]:
def eval_condition(iepoch,print_result_every_x_epoch):
    if (iepoch + 1) % print_result_every_x_epoch == 0:
        return True
    else:
        return False


def eval_model(model, dataloader):
    predict_list = np.array([])
    label_list = np.array([])
    for sample in dataloader:
        y_predict = model(sample[0])
        y_predict = y_predict.detach().cpu().numpy()
        y_predict = np.argmax(y_predict, axis=1)
        predict_list = np.concatenate((predict_list, y_predict), axis=0)
        label_list = np.concatenate((label_list, sample[1].detach().cpu().numpy()), axis=0)
    acc = accuracy_score(predict_list, label_list)
    return acc


def save_to_log(sentence, Result_log_folder, dataset_name):
    father_path = Result_log_folder + dataset_name
    if not os.path.exists(father_path):
        os.makedirs(father_path)
    path = father_path + '/' + dataset_name + '_.txt'
    print(path)
    with open(path, "a") as myfile:
        myfile.write(sentence + '\n')
        
        
def get_Prime_number_in_a_range(start, end):
    Prime_list = []
    for val in range(start, end + 1): 
        prime_or_not = True
        for n in range(2, val):
            if (val % n) == 0:
                prime_or_not = False
                break
        if prime_or_not:
            Prime_list.append(val)
    return Prime_list


def get_out_channel_number(paramenter_layer, in_channel, prime_list):
    out_channel_expect = int(paramenter_layer/(in_channel*sum(prime_list)))
    return out_channel_expect

def generate_layer_parameter_list(start,end,paramenter_number_of_layer_list, in_channel = 1):
    prime_list = get_Prime_number_in_a_range(start, end)
    if prime_list == []:
        print('start = ',start, 'which is larger than end = ', end)
    paramenter_number_of_layer_list[0] =  paramenter_number_of_layer_list[0]*in_channel
    input_in_channel = in_channel
    layer_parameter_list = []
    for paramenter_number_of_layer in paramenter_number_of_layer_list:
        out_channel = get_out_channel_number(paramenter_number_of_layer, in_channel, prime_list)
        
        tuples_in_layer= []
        for prime in prime_list:
            tuples_in_layer.append((in_channel,out_channel,prime))
        in_channel =  len(prime_list)*out_channel
        
        layer_parameter_list.append(tuples_in_layer)
    
    tuples_in_layer_last = []
    first_out_channel = len(prime_list)*get_out_channel_number(paramenter_number_of_layer_list[0], input_in_channel, prime_list)
    tuples_in_layer_last.append((in_channel,first_out_channel,start))
    tuples_in_layer_last.append((in_channel,first_out_channel,start+1))
    layer_parameter_list.append(tuples_in_layer_last)
    return layer_parameter_list

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import copy

def calculate_mask_index(kernel_length_now,largest_kernel_lenght):
    right_zero_mast_length = math.ceil((largest_kernel_lenght-1)/2)-math.ceil((kernel_length_now-1)/2)
    left_zero_mask_length = largest_kernel_lenght - kernel_length_now - right_zero_mast_length
    return left_zero_mask_length, left_zero_mask_length+ kernel_length_now

def creat_mask(number_of_input_channel,number_of_output_channel, kernel_length_now, largest_kernel_lenght):
    ind_left, ind_right= calculate_mask_index(kernel_length_now,largest_kernel_lenght)
    mask = np.ones((number_of_input_channel,number_of_output_channel,largest_kernel_lenght))
    mask[:,:,0:ind_left]=0
    mask[:,:,ind_right:]=0
    return mask


def creak_layer_mask(layer_parameter_list):
    largest_kernel_lenght = layer_parameter_list[-1][-1]
    mask_list = []
    init_weight_list = []
    bias_list = []
    for i in layer_parameter_list:
        conv = torch.nn.Conv1d(in_channels=i[0], out_channels=i[1], kernel_size=i[2])
        ind_l,ind_r= calculate_mask_index(i[2],largest_kernel_lenght)
        big_weight = np.zeros((i[1],i[0],largest_kernel_lenght))
        big_weight[:,:,ind_l:ind_r]= conv.weight.detach().numpy()
        
        bias_list.append(conv.bias.detach().numpy())
        init_weight_list.append(big_weight)
        
        mask = creat_mask(i[1],i[0],i[2], largest_kernel_lenght)
        mask_list.append(mask)
        
    mask = np.concatenate(mask_list, axis=0)
    init_weight = np.concatenate(init_weight_list, axis=0)
    init_bias = np.concatenate(bias_list, axis=0)
    return mask.astype(np.float32), init_weight.astype(np.float32), init_bias.astype(np.float32)

    
class build_layer_with_layer_parameter(nn.Module):
    def __init__(self,layer_parameters):
        super(build_layer_with_layer_parameter, self).__init__()

        os_mask, init_weight, init_bias= creak_layer_mask(layer_parameters)
        
        
        in_channels = os_mask.shape[1] 
        out_channels = os_mask.shape[0] 
        max_kernel_size = os_mask.shape[-1]

        self.weight_mask = nn.Parameter(torch.from_numpy(os_mask),requires_grad=False)
        
        self.padding = nn.ConstantPad1d((int((max_kernel_size-1)/2), int(max_kernel_size/2)), 0)
         
        self.conv1d = torch.nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=max_kernel_size)
        self.conv1d.weight = nn.Parameter(torch.from_numpy(init_weight),requires_grad=True)
        self.conv1d.bias =  nn.Parameter(torch.from_numpy(init_bias),requires_grad=True)

        self.bn = nn.BatchNorm1d(num_features=out_channels)
    
    def forward(self, X):
        self.conv1d.weight.data = self.conv1d.weight*self.weight_mask
        #self.conv1d.weight.data.mul_(self.weight_mask)
        result_1 = self.padding(X)
        result_2 = self.conv1d(result_1)
        result_3 = self.bn(result_2)
        result = F.relu(result_3)
        return result    
    
    
class OS_CNN_block(nn.Module):
    def __init__(self,layer_parameter_list,n_class,squeeze_layer = True):
        super(OS_CNN_block, self).__init__()
        self.squeeze_layer = squeeze_layer
        self.layer_parameter_list = layer_parameter_list
        self.layer_list = []
        
        
        for i in range(len(layer_parameter_list)):
            layer = build_layer_with_layer_parameter(layer_parameter_list[i])
            self.layer_list.append(layer)
        
        self.net = nn.Sequential(*self.layer_list)
            
        self.averagepool = nn.AdaptiveAvgPool1d(1)
        
        out_put_channel_numebr = 0
        for final_layer_parameters in layer_parameter_list[-1]:
            out_put_channel_numebr = out_put_channel_numebr+ final_layer_parameters[1] 
            
        self.hidden = nn.Linear(out_put_channel_numebr, n_class)

    def forward(self, X):
        
        X = self.net(X)
        
        if self.squeeze_layer:
            X = self.averagepool(X)
            X = X.squeeze_(-1)
            X = self.hidden(X)
        return X

def check_channel_limit(os_block_layer_parameter_list,n_input_channel,mid_channel_limit): 
    out_channel_each = 0
    for conv_in in os_block_layer_parameter_list[-1]:
        out_channel_each = out_channel_each + conv_in[1]
    total_temp_channel = n_input_channel*out_channel_each
    if total_temp_channel<=mid_channel_limit:
        return os_block_layer_parameter_list
    else:
        
        temp_channel_each = max(int(mid_channel_limit/(n_input_channel*len(os_block_layer_parameter_list[-1]))),1)
        for i in range(len(os_block_layer_parameter_list[-1])):
            os_block_layer_parameter_list[-1][i]= (os_block_layer_parameter_list[-1][i][0],
                                                   temp_channel_each,
                                                  os_block_layer_parameter_list[-1][i][2])
        print('reshape temp channel from ',total_temp_channel,' to ',n_input_channel,' * ',temp_channel_each,)
        return os_block_layer_parameter_list

    
class OS_CNN(nn.Module):
    def __init__(self, layer_parameter_list, n_class, n_input_channel,squeeze_layer = True):
        super(OS_CNN, self).__init__()
        
        self.mid_channel_limit = 1000
        self.squeeze_layer = squeeze_layer
        self.layer_parameter_list = layer_parameter_list
        self.OS_block_list = nn.ModuleList()
        
        os_block_layer_parameter_list = copy.deepcopy(layer_parameter_list[:-1])
        os_block_layer_parameter_list = check_channel_limit(os_block_layer_parameter_list,n_input_channel,self.mid_channel_limit)
        print('os_block_layer_parameter_list is :',os_block_layer_parameter_list)
        for nth in range(n_input_channel):
            torch_OS_CNN_block = OS_CNN_block(os_block_layer_parameter_list,n_class, False)
            self.OS_block_list.append(torch_OS_CNN_block)
        
        rf_size = layer_parameter_list[0][-1][-1]
        in_channel_we_want= len(layer_parameter_list[1])*os_block_layer_parameter_list[-1][-1][1]*n_input_channel
        print('in_channel_we_want is :', in_channel_we_want)
       
        layer_parameter_list = generate_layer_parameter_list(1,rf_size+1,[8*128, (5*128*256 + 2*256*128)/2],in_channel = in_channel_we_want)
        
        self.averagepool = nn.AdaptiveAvgPool1d(1) 
        print(layer_parameter_list)
        self.OS_net =  OS_CNN_block(layer_parameter_list,n_class, True)

    def forward(self, X):
        OS_block_result_list = []
        for i_th_channel, OS_block in enumerate(self.OS_block_list):
            OS_block_result = OS_block(X[:,i_th_channel:i_th_channel+1,:])
            OS_block_result_list.append(OS_block_result)
        result = F.relu(torch.cat(tuple(OS_block_result_list), 1)) 
        
        result = self.OS_net(result)
        return result

        

In [6]:
import os
from sklearn.metrics import accuracy_score
from os.path import dirname
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset



class OS_CNN_easy_use():
    
    def __init__(self,Result_log_folder, 
                 dataset_name, 
                 device, 
                 Max_kernel_size = 89, 
                 paramenter_number_of_layer_list = [8*128, 5*128*256 + 2*256*128], 
                 max_epoch = 2000, 
                 batch_size=16,
                 print_result_every_x_epoch = 50,
                 start_kernel_size = 1,
                 quarter_or_half = 4
                ):
        
        super(OS_CNN_easy_use, self).__init__()
        
        if not os.path.exists(Result_log_folder +dataset_name+'/'):
            os.makedirs(Result_log_folder +dataset_name+'/')
        Initial_model_path = Result_log_folder +dataset_name+'/'+dataset_name+'initial_model'
        model_save_path = Result_log_folder +dataset_name+'/'+dataset_name+'Best_model'
        

        self.Result_log_folder = Result_log_folder
        self.dataset_name = dataset_name        
        self.model_save_path = model_save_path
        self.Initial_model_path = Initial_model_path
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")
        
        
        self.Max_kernel_size = Max_kernel_size
        self.paramenter_number_of_layer_list = paramenter_number_of_layer_list
        self.max_epoch = max_epoch
        self.batch_size = batch_size
        self.print_result_every_x_epoch = print_result_every_x_epoch
        self.quarter_or_half = quarter_or_half
        
        self.OS_CNN = None
        
        
        self.start_kernel_size = start_kernel_size
        
    def fit(self, X_train, y_train, X_val, y_val):

        print('code is running on ',self.device)
        
        # covert numpy to pytorch tensor and put into gpu
        X_train = torch.from_numpy(X_train)
        X_train.requires_grad = False
        if len(X_train.shape) ==3:
            X_train = X_train.to(self.device)
        else:
            X_train = X_train.unsqueeze_(1).to(self.device)
        y_train = torch.from_numpy(y_train).to(self.device)
        n_input_channel = X_train.shape[1]
        
        X_test = torch.from_numpy(X_val)
        X_test.requires_grad = False
        if len(X_test.shape) ==3:
            X_test = X_test.to(self.device)
        else:
            X_test = X_test.unsqueeze_(1).to(self.device)
        y_test = torch.from_numpy(y_val).to(self.device)
        
        
        input_shape = X_train.shape[-1]
        n_class = max(y_train) + 1
        receptive_field_shape= min(int(X_train.shape[-1]/self.quarter_or_half),self.Max_kernel_size)
        
        # generate parameter list
        layer_parameter_list = generate_layer_parameter_list(self.start_kernel_size,receptive_field_shape,self.paramenter_number_of_layer_list,in_channel = 1)
        print(layer_parameter_list)
        torch_OS_CNN = OS_CNN(layer_parameter_list, n_class.item(),n_input_channel, True).to(self.device)
        
        # save_initial_weight
        torch.save(torch_OS_CNN.state_dict(), self.Initial_model_path)
        
        
        # loss, optimizer, scheduler
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(torch_OS_CNN.parameters())
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=200, min_lr=0.0001)
        
        # build dataloader
        print(max(int(min(X_train.shape[0] / 10, self.batch_size)),2))
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=max(int(min(X_train.shape[0] / 10, self.batch_size)),2), shuffle=True)
        test_dataset = TensorDataset(X_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=max(int(min(X_train.shape[0] / 10, self.batch_size)),2), shuffle=False)
        
        
        torch_OS_CNN.train()   
        
        for i in range(self.max_epoch):
            for sample in train_loader:
                optimizer.zero_grad()
                y_predict = torch_OS_CNN(sample[0])
                output = criterion(y_predict, sample[1])
                output.backward()
                optimizer.step()
            scheduler.step(output)
            
            if eval_condition(i,self.print_result_every_x_epoch):
                for param_group in optimizer.param_groups:
                    print('epoch =',i, 'lr = ', param_group['lr'])
                torch_OS_CNN.eval()
                acc_train = eval_model(torch_OS_CNN, train_loader)
                acc_test = eval_model(torch_OS_CNN, test_loader)
                torch_OS_CNN.train()
                print('train_acc=\t', acc_train, '\t test_acc=\t', acc_test, '\t loss=\t', output.item())
                sentence = 'train_acc=\t'+str(acc_train)+ '\t test_acc=\t'+str(acc_test) 
                print('log saved at:')
                save_to_log(sentence,self.Result_log_folder, self.dataset_name)
                torch.save(torch_OS_CNN.state_dict(), self.model_save_path)
         
        torch.save(torch_OS_CNN.state_dict(), self.model_save_path)
        self.OS_CNN = torch_OS_CNN

        
        
    def predict(self, X_test):
        
        X_test = torch.from_numpy(X_test)
        X_test.requires_grad = False
        if len(X_test.shape) ==3:
            X_test = X_test.to(self.device)
        else:
            X_test = X_test.unsqueeze_(1).to(self.device)
        
        test_dataset = TensorDataset(X_test)
        test_loader = DataLoader(test_dataset, batch_size=int(min(X_test.shape[0] / 10, self.batch_size)), shuffle=False)
        
        self.OS_CNN.eval()
        
        predict_list = np.array([])
        for sample in test_loader:
            y_predict = self.OS_CNN(sample[0])
            y_predict = y_predict.detach().cpu().numpy()
            y_predict = np.argmax(y_predict, axis=1)
            predict_list = np.concatenate((predict_list, y_predict), axis=0)
            
        return predict_list

In [7]:
def max_pooling_data(a,sampling):
    a = np.float32(a)
    a_torch = torch.from_numpy(a)
    a_torch.unsqueeze_(1)
    m = nn.MaxPool1d(sampling, stride=sampling)
    output = m(a_torch)
    return output.squeeze_(1).numpy()

def down_sampling_data(a,sampling):
    return a[:,::sampling]

def long_data_to_more_channels(a, channels):
    more_length_required = channels - a.shape[-1]%channels
    padding_size = list(a.shape)
    padding_size[-1]= int(more_length_required)
    padding = np.zeros(tuple(padding_size))
    a = np.concatenate((a, padding), axis=-1)
    result = 1
    for i in a.shape[1:]:
        result = result*i
    result = result*channels/a.shape[-1]
    z = np.swapaxes(np.reshape(a,(a.shape[0],int(a.shape[-1]/channels),int(result))),1,2)
    return np.float32(z)

In [8]:
dataset_name_list = [
'ArticularyWordRecognition',
# 'AtrialFibrillation',
# 'BasicMotions',
# 'CharacterTrajectories',
# 'Cricket',
# 'DuckDuckGeese',
# 'EigenWorms', 
# 'Epilepsy',
# 'ERing',
# 'EthanolConcentration',
# 'FaceDetection',
# 'FingerMovements',
# 'HandMovementDirection',
# 'Handwriting',
# 'Heartbeat',
# 'InsectWingbeat',# to long to run
# 'JapaneseVowels',
# 'Libras',
# 'LSST',
# 'MotorImagery',
# 'NATOPS',
# 'PEMS-SF',
# 'PenDigits',
# 'PhonemeSpectra',
# 'RacketSports',
# 'SelfRegulationSCP1',
# 'SelfRegulationSCP2',
# 'SpokenArabicDigits',
# 'StandWalkJump',
# 'UWaveGestureLibrary'
]

In [9]:
import os
from os.path import dirname
from sklearn.metrics import accuracy_score


Result_log_folder = './Results_of_OS_OS_CNN_for_UEAArchive_2018/OS_CNN_result_iter_0/'
dataset_path = dirname("./Example_Datasets/UEAArchive_2018/")


for dataset_name in dataset_name_list:
    print('running at:', dataset_name)   
    # load data
    X_train, y_train, X_test, y_test = TSC_multivariate_data_loader(dataset_path, dataset_name)
    print(X_train.shape)
    model = OS_CNN_easy_use(
        Result_log_folder = Result_log_folder, # the Result_log_folder
        dataset_name = dataset_name,           # dataset_name for creat log under Result_log_folder
        device = "cuda:0",                     # Gpu 
        max_epoch = 100,                       # In our expirement the number is 2000 for keep it same with FCN for the example dataset 500 will be enough
        Max_kernel_size = 89, 
        start_kernel_size = 1,
        paramenter_number_of_layer_list = [8*128, (5*128*256 + 2*256*128)/2], 
        quarter_or_half = 4,
        )
    
    model.fit(X_train, y_train, X_test, y_test)
    
    
    
    y_predict = model.predict(X_test)
    
    
    print('correct:',y_test)
    print('predict:',y_predict)
    acc = accuracy_score(y_predict, y_test)
    print(acc)
    


running at: ArticularyWordRecognition
(275, 9, 144)
code is running on  cuda:0
[[(1, 6, 1), (1, 6, 2), (1, 6, 3), (1, 6, 5), (1, 6, 7), (1, 6, 11), (1, 6, 13), (1, 6, 17), (1, 6, 19), (1, 6, 23), (1, 6, 29), (1, 6, 31)], [(72, 9, 1), (72, 9, 2), (72, 9, 3), (72, 9, 5), (72, 9, 7), (72, 9, 11), (72, 9, 13), (72, 9, 17), (72, 9, 19), (72, 9, 23), (72, 9, 29), (72, 9, 31)], [(108, 72, 1), (108, 72, 2)]]
os_block_layer_parameter_list is : [[(1, 6, 1), (1, 6, 2), (1, 6, 3), (1, 6, 5), (1, 6, 7), (1, 6, 11), (1, 6, 13), (1, 6, 17), (1, 6, 19), (1, 6, 23), (1, 6, 29), (1, 6, 31)], [(72, 9, 1), (72, 9, 2), (72, 9, 3), (72, 9, 5), (72, 9, 7), (72, 9, 11), (72, 9, 13), (72, 9, 17), (72, 9, 19), (72, 9, 23), (72, 9, 29), (72, 9, 31)]]
in_channel_we_want is : 972
[[(972, 6, 1), (972, 6, 2), (972, 6, 3), (972, 6, 5), (972, 6, 7), (972, 6, 11), (972, 6, 13), (972, 6, 17), (972, 6, 19), (972, 6, 23), (972, 6, 29), (972, 6, 31)], [(72, 9, 1), (72, 9, 2), (72, 9, 3), (72, 9, 5), (72, 9, 7), (72, 9, 11)